In [1]:
!pyenv global 3.6
!eval "$(pyenv init --path)"
!python3 --version

Python 3.6.4


In [2]:
# !git clone https://github.com/Samsung-IT-Academy/stepik-dl-nlp.git && pip install -r stepik-dl-nlp/requirements.txt
import sys; sys.path.append('./stepik-dl-nlp')

import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import accuracy_score

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import collections

import torch
from torch import nn
from torch.nn import functional as F

import sys
import os
from tqdm import tqdm  
import pandas as pd
from sklearn.model_selection import train_test_split
from gensim import utils
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from gensim.models import Word2Vec
import gensim.downloader as api

# import dlnlputils
# from dlnlputils.data import tokenize_text_simple_regex, tokenize_corpus, build_vocabulary, \
#     vectorize_texts, SparseFeaturesDataset
# from dlnlputils.pipeline import train_eval_loop, predict_with_model, init_random_seed

# init_random_seed()

In [3]:
import numpy as np
import scipy.sparse
import torch
from torch.utils.data import Dataset

In [ ]:
#подготовка признаков

In [4]:
import collections
import re

import numpy as np

TOKEN_RE = re.compile(r'[\w\d]+')


def tokenize_text_simple_regex(txt, min_token_size=4):
    txt = txt.lower()
    all_tokens = TOKEN_RE.findall(txt)
    return [token for token in all_tokens if len(token) >= min_token_size]


def character_tokenize(txt):
    return list(txt)


def tokenize_corpus(texts, tokenizer=tokenize_text_simple_regex, **tokenizer_kwargs):
    return [tokenizer(text, **tokenizer_kwargs) for text in texts]


def add_fake_token(word2id, token=''):
    word2id_new = {token: i + 1 for token, i in word2id.items()}
    word2id_new[token] = 0
    return word2id_new


def texts_to_token_ids(tokenized_texts, word2id):
    return [[word2id[token] for token in text if token in word2id]
            for text in tokenized_texts]


def build_vocabulary(tokenized_texts, max_size=1000000, max_doc_freq=0.8, min_count=5, pad_word=None):
    word_counts = collections.defaultdict(int)
    doc_n = 0

    # посчитать количество документов, в которых употребляется каждое слово
    # а также общее количество документов
    for txt in tokenized_texts:
        doc_n += 1
        unique_text_tokens = set(txt)
        for token in unique_text_tokens:
            word_counts[token] += 1

    # убрать слишком редкие и слишком частые слова
    word_counts = {word: cnt for word, cnt in word_counts.items()
                   if cnt >= min_count and cnt / doc_n <= max_doc_freq}

    # отсортировать слова по убыванию частоты
    sorted_word_counts = sorted(word_counts.items(),
                                reverse=True,
                                key=lambda pair: pair[1])

    # добавим несуществующее слово с индексом 0 для удобства пакетной обработки
    if pad_word is not None:
        sorted_word_counts = [(pad_word, 0)] + sorted_word_counts

    # если у нас по прежнему слишком много слов, оставить только max_size самых частотных
    if len(word_counts) > max_size:
        sorted_word_counts = sorted_word_counts[:max_size]

    # нумеруем слова
    word2id = {word: i for i, (word, _) in enumerate(sorted_word_counts)}

    # нормируем частоты слов
    word2freq = np.array([cnt / doc_n for _, cnt in sorted_word_counts], dtype='float32')

    return word2id, word2freq


PAD_TOKEN = '__PAD__'
NUMERIC_TOKEN = '__NUMBER__'
NUMERIC_RE = re.compile(r'^([0-9.,e+\-]+|[mcxvi]+)$', re.I)


def replace_number_nokens(tokenized_texts):
    return [[token if not NUMERIC_RE.match(token) else NUMERIC_TOKEN for token in text]
            for text in tokenized_texts]

In [5]:
import numpy as np
import scipy.sparse
import torch
from torch.utils.data import Dataset


def vectorize_texts(tokenized_texts, word2id, word2freq, mode='tfidf', scale=True):
    assert mode in {'tfidf', 'idf', 'tf', 'bin'}

    # считаем количество употреблений каждого слова в каждом документе
    result = scipy.sparse.dok_matrix((len(tokenized_texts), len(word2id)), dtype='float32')
    for text_i, text in enumerate(tokenized_texts):
        for token in text:
            if token in word2id:
                result[text_i, word2id[token]] += 1

    # получаем бинарные вектора "встречается или нет"
    if mode == 'bin':
        result = (result > 0).astype('float32')

    # получаем вектора относительных частот слова в документе
    elif mode == 'tf':
        result = result.tocsr()
        result = result.multiply(1 / result.sum(1))

    # полностью убираем информацию о количестве употреблений слова в данном документе,
    # но оставляем информацию о частотности слова в корпусе в целом
    elif mode == 'idf':
        result = (result > 0).astype('float32').multiply(1 / word2freq)

    # учитываем всю информацию, которая у нас есть:
    # частоту слова в документе и частоту слова в корпусе
    elif mode == 'tfidf':
        result = result.tocsr()
        result = result.multiply(1 / result.sum(1))  # разделить каждую строку на её длину
        result = result.multiply(1 / word2freq)  # разделить каждый столбец на вес слова

    if scale:
        result = result.tocsc()
        result -= result.min()
        result /= (result.max() + 1e-6)

    return result.tocsr()


class SparseFeaturesDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets

    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, idx):
        cur_features = torch.from_numpy(self.features[idx].toarray()[0]).float()
        cur_label = torch.from_numpy(np.asarray(self.targets[idx])).long()
        return cur_features, cur_label

In [ ]:
import pandas as pd
df = pd.read_csv("data/bitkogan.csv")

In [ ]:
df = pd.DataFrame([])
for root, dirs, files in os.walk('data'):
    for file in tqdm(files[:1]):
        df1 = pd.read_csv(f'data/{file}')
        df = pd.concat([df, df1], axis=0, ignore_index=True)

df = df.sort_values('date', ignore_index=True)
df.head()

In [15]:
pnl = pd.read_csv('Backtest_USDRUB_5_days.txt')
pnl['date_start'] = pd.to_datetime(pnl['date_start']).dt.strftime('%Y-%m-%d')
pnl['pnl_sign'] = pnl['pnl'].apply(lambda x: 1 if x >= 0 else 0)
pnl

,date_start,pnl,pnl_sign
0,2021-01-04,291264.097914,1
1,2021-01-05,-411993.830320,0
2,2021-01-06,521491.686795,1
3,2021-01-08,62842.634116,1
4,2021-01-11,-537598.706217,0
...,...,...,...
245,2021-12-20,-407527.554561,0
246,2021-12-21,-176881.417077,0
247,2021-12-22,-206943.414418,0
248,2021-12-23,23073.596468,1


In [16]:
pnl['pnl_sign'].sum() / pnl.shape[0]

0.536

In [ ]:
pnl['cumm_text'] = None
pnl

In [ ]:
df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')

In [ ]:
df_with_pnl = df[(df['date'] >= '2021-01-01') & (df['date'] <= '2021-12-31')]
df_with_pnl

In [ ]:
i = 0
texts = ''
for _, row in df_with_pnl.iterrows():
    if i >= pnl.shape[0]:
        break

    if row['date'] <= pnl['date_start'][i]:
        texts += ' ' + row['text']
    else:
        pnl.at[i, 'cumm_text'] = texts
        texts = ''
        i += 1
        continue

pnl.head()

In [ ]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(pnl)

In [ ]:
train_tokenized = tokenize_corpus(train['cumm_text'])
val_tokenized = tokenize_corpus(val['cumm_text'])

In [ ]:
print(' '.join(train_tokenized[0]))

In [ ]:
# строим словарь - vocabulary с помощью функции build_vocabulary
# принимает на вход список списков токенезированные
# word_doc_freq - содержит относительные частоты всех слов в датасете, он понадобиться 
# на этапе формирования матрицы признаков

MAX_DF = 0.8 #во скольких документах встречаеться слово
MIN_COUNT = 5 # сколько раз слово встречаеться в тексте


vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
UNIQUE_WORDS_N = len(vocabulary)
print('Количество уникальных токенов', UNIQUE_WORDS_N)
print(list(vocabulary.items())[:10])

In [ ]:
plt.hist(word_doc_freq, bins=20)
plt.title('Распределение относительных частот слов')
plt.yscale('log');

In [ ]:
VECTORIZATION_MODE = 'tfidf'
# построение матрицы признаков по методу мешка слов
# функция vectorize_texts принимает на вход
#1. токенизированные список списков
#2. словарь
#3. вектор частоты токенизированны
#4. алгоритм взвешивания токенов по частоте mode - есть 4 алгорима - bin,tf,idf,tfidf
#5. флаг чтобы перемаштабировать флаг после взвешивания

train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
print()
print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
print()


In [ ]:
plt.hist(train_vectors.data, bins=20)
plt.title('Распределение весов признаков')
plt.yscale('log');

In [ ]:
#word2vec

In [ ]:
from gensim.models import Word2Vec

In [ ]:
train_vectors

In [ ]:
import gensim.downloader as api
for k, v in api.info().items():
    for k1, v1 in v.items():
        print(k1)

In [ ]:
model_t1 = api.load('glove-twitter-200')

In [10]:
model_t = api.load('glove-twitter-100')

In [11]:
from gensim import utils

def get_vectors_gt100(row):
    '''
      word_doc_freq # частоты слов
      train_tokenized #сами слова
    '''
    vecs = [np.zeros(100)]
    for word in row:
        #print(row)
        try: 
            # если слово есть в нашем очищенном словаре
            # умножаем вектор на вес tfidf
            v = model_t[word] * word_doc_freq[vocabulary[word]] 
        except:
            v = np.zeros(100)
        vecs.append(v)
    return np.sum(np.array(vecs),axis=0)

In [ ]:
train_vectors_gt100 = np.array([get_vectors_gt100(i) for i in train_tokenized])
val_vectors_gt100 = np.array([get_vectors_gt100(i) for i in val_tokenized])

In [ ]:
train_vectors_gt100, train_vectors_gt100.shape

In [ ]:
model_t1.most_similar(positive=['инвестор', 'рынок'], topn=1)

In [9]:
model_t.most_similar(positive=['инвестор', 'рынок'], topn=1)

NameError: name 'model_t' is not defined

In [ ]:
y_train = train['pnl_sign']
X_train = train_vectors_gt100

y_val = val['pnl_sign']
X_val = val_vectors_gt100

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)
print(model.score(X_val, y_val))
print('данные:', list(os.walk('data/'))[0][-1][:-2])

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)
print(model.score(X_val, y_val))
print('данные:', list(os.walk('data/'))[0][-1][:-2])

In [7]:
from gensim import utils

def get_vectors_gt100(row):
    '''
      word_doc_freq # частоты слов
      train_tokenized #сами слова
    '''
    vecs = [np.zeros(100)]
    for word in row:
        #print(row)
        try: 
            # если слово есть в нашем очищенном словаре
            # умножаем вектор на вес tfidf
            v = model_t[word] * word_doc_freq[vocabulary[word]] 
        except:
            v = np.zeros(100)
        vecs.append(v)
    return np.sum(np.array(vecs),axis=0)

In [ ]:
model_t = api.load('glove-twitter-100')

In [12]:
input_path = 'data/' # нужно указать путь до папки с данными по новостям

for root, dirs, files in os.walk(input_path):
    for q in tqdm(range(1, 12)):
        df = pd.DataFrame([])
        for file in files[:q]:
            df1 = pd.read_csv(f'{input_path}/{file}')
            df = pd.concat([df, df1], axis=0, ignore_index=True)

        df = df.sort_values('date', ignore_index=True)
        df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')

        pnl = pd.read_csv('Backtest_USDRUB_5_days.txt')
        pnl['date_start'] = pd.to_datetime(pnl['date_start']).dt.strftime('%Y-%m-%d')
        pnl['pnl_sign'] = pnl['pnl'].apply(lambda x: 1 if x >= 0 else 0)
        pnl['cumm_text'] = None

        df_with_pnl = df[(df['date'] >= '2021-01-01') & (df['date'] <= '2021-12-31')]

        i = 0
        texts = ''
        for _,row in df_with_pnl.iterrows():
            if i >= pnl.shape[0]:
                break

            if row['date'] <= pnl['date_start'][i]:
                texts += ' ' + row['text']
            else:
                pnl.at[i,'cumm_text'] = texts
                texts = ''
                i += 1
                continue

        train, val = train_test_split(pnl)
        train_tokenized = tokenize_corpus(train['cumm_text'])
        val_tokenized = tokenize_corpus(val['cumm_text'])

        MAX_DF = 0.8 #во скольких документах встречаеться слово
        MIN_COUNT = 5 # сколько раз слово встречаеться в тексте


        vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
        UNIQUE_WORDS_N = len(vocabulary)    
        VECTORIZATION_MODE = 'tfidf'
        train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

        train_vectors_gt100 = np.array([get_vectors_gt100(i) for i in train_tokenized])
        val_vectors_gt100 = np.array([get_vectors_gt100(i) for i in val_tokenized])

        model_t.most_similar(positive=['инвестор', 'рынок'], topn=1)

        y_train = train['pnl_sign']
        X_train = train_vectors_gt100

        y_val = val['pnl_sign']
        X_val = val_vectors_gt100

        model = LogisticRegression()
        model.fit(X_train, y_train)
        print(model.score(X_val, y_val))
        print('данные:', files[:q])
        print('всего наблюдений:', df_with_pnl.shape[0])


  9%|████                                        | 1/11 [00:02<00:22,  2.25s/it]

0.5238095238095238
данные: ['cbonds.csv']
всего наблюдений: 3618


 18%|████████                                    | 2/11 [00:04<00:19,  2.11s/it]

0.5714285714285714
данные: ['cbonds.csv', 'themovchans.csv']
всего наблюдений: 3726


 27%|████████████                                | 3/11 [00:06<00:16,  2.04s/it]

...

0.38095238095238093
данные: ['cbonds.csv', 'themovchans.csv', 'headlines_QUANTS.csv', 'War_Wealth_Wisdom.csv', 'mmi.csv', 'vts.csv', 'signal.csv', '.gitignore', 'zgmail.csv']
всего наблюдений: 20494


 91%|███████████████████████████████████████    | 10/11 [00:38<00:05,  5.19s/it]

0.49206349206349204
данные: ['cbonds.csv', 'themovchans.csv', 'headlines_QUANTS.csv', 'War_Wealth_Wisdom.csv', 'mmi.csv', 'vts.csv', 'signal.csv', '.gitignore', 'zgmail.csv', 'rshb_invest.csv']
всего наблюдений: 22043


100%|███████████████████████████████████████████| 11/11 [00:44<00:00,  4.06s/it]

0.47619047619047616
данные: ['cbonds.csv', 'themovchans.csv', 'headlines_QUANTS.csv', 'War_Wealth_Wisdom.csv', 'mmi.csv', 'vts.csv', 'signal.csv', '.gitignore', 'zgmail.csv', 'rshb_invest.csv', 'Alfa_Wealth.csv']
всего наблюдений: 22218


In [13]:
input_path = 'data/'

for root, dirs, files in os.walk(input_path):
    for q in tqdm(range(1, 12)):
        df = pd.DataFrame([])
        for file in files[:q]:
            df1 = pd.read_csv(f'{input_path}/{file}')
            df = pd.concat([df, df1], axis=0, ignore_index=True)

        df = df.sort_values('date', ignore_index=True)
        df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')

        pnl = pd.read_csv('Backtest_USDRUB_5_days.txt')
        pnl['date_start'] = pd.to_datetime(pnl['date_start']).dt.strftime('%Y-%m-%d')
        pnl['pnl_sign'] = pnl['pnl'].apply(lambda x: 1 if x >= 0 else 0)
        pnl['cumm_text'] = None

        df_with_pnl = df[(df['date'] >= '2021-01-01') & (df['date'] <= '2021-12-31')]

        i = 0
        texts = ''
        for _,row in df_with_pnl.iterrows():
            if i >= pnl.shape[0]:
                break

            if row['date'] <= pnl['date_start'][i]:
                texts += ' ' + row['text']
            else:
                pnl.at[i,'cumm_text'] = texts
                texts = ''
                i += 1
                continue

        train, val = train_test_split(pnl)
        train_tokenized = tokenize_corpus(train['cumm_text'])
        val_tokenized = tokenize_corpus(val['cumm_text'])

        MAX_DF = 0.8 #во скольких документах встречаеться слово
        MIN_COUNT = 5 # сколько раз слово встречаеться в тексте


        vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
        UNIQUE_WORDS_N = len(vocabulary)    
        VECTORIZATION_MODE = 'tfidf'
        train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

        train_vectors_gt100 = np.array([get_vectors_gt100(i) for i in train_tokenized])
        val_vectors_gt100 = np.array([get_vectors_gt100(i) for i in val_tokenized])

        model_t.most_similar(positive=['инвестор', 'рынок'], topn=1)

        y_train = train['pnl_sign']
        X_train = train_vectors_gt100

        y_val = val['pnl_sign']
        X_val = val_vectors_gt100

        model = LogisticRegression()
        model.fit(X_train, y_train)
        print(model.score(X_val, y_val))
        print('данные:', files[:q])
        print('всего наблюдений:', df_with_pnl.shape[0])


  9%|████                                        | 1/11 [00:02<00:20,  2.01s/it]

0.42857142857142855
данные: ['cbonds.csv']
всего наблюдений: 3618


 18%|████████                                    | 2/11 [00:04<00:18,  2.04s/it]

0.5079365079365079
данные: ['cbonds.csv', 'themovchans.csv']
всего наблюдений: 3726


 27%|████████████                                | 3/11 [00:06<00:16,  2.01s/it]

...

0.4126984126984127
данные: ['cbonds.csv', 'themovchans.csv', 'headlines_QUANTS.csv', 'War_Wealth_Wisdom.csv', 'mmi.csv', 'vts.csv', 'signal.csv', '.gitignore', 'zgmail.csv']
всего наблюдений: 20494


 91%|███████████████████████████████████████    | 10/11 [00:38<00:05,  5.17s/it]

0.5873015873015873
данные: ['cbonds.csv', 'themovchans.csv', 'headlines_QUANTS.csv', 'War_Wealth_Wisdom.csv', 'mmi.csv', 'vts.csv', 'signal.csv', '.gitignore', 'zgmail.csv', 'rshb_invest.csv']
всего наблюдений: 22043


100%|███████████████████████████████████████████| 11/11 [00:44<00:00,  4.05s/it]

0.30158730158730157
данные: ['cbonds.csv', 'themovchans.csv', 'headlines_QUANTS.csv', 'War_Wealth_Wisdom.csv', 'mmi.csv', 'vts.csv', 'signal.csv', '.gitignore', 'zgmail.csv', 'rshb_invest.csv', 'Alfa_Wealth.csv']
всего наблюдений: 22218


In [14]:
input_path = 'data/'

for root, dirs, files in os.walk(input_path):
    for q in tqdm(range(1, 12)):
        df = pd.DataFrame([])
        for file in files[:q]:
            df1 = pd.read_csv(f'{input_path}/{file}')
            df = pd.concat([df, df1], axis=0, ignore_index=True)

        df = df.sort_values('date', ignore_index=True)
        df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')

        pnl = pd.read_csv('Backtest_USDRUB_5_days.txt')
        pnl['date_start'] = pd.to_datetime(pnl['date_start']).dt.strftime('%Y-%m-%d')
        pnl['pnl_sign'] = pnl['pnl'].apply(lambda x: 1 if x >= 0 else 0)
        pnl['cumm_text'] = None

        df_with_pnl = df[(df['date'] >= '2021-01-01') & (df['date'] <= '2021-12-31')]

        i = 0
        texts = ''
        for _,row in tqdm(df_with_pnl.iterrows()):
            if i >= pnl.shape[0]:
                break

            if row['date'] <= pnl['date_start'][i]:
                texts += ' ' + row['text']
            else:
                pnl.at[i,'cumm_text'] = texts
                texts = ''
                i += 1
                continue

        train, val = train_test_split(pnl)
        train_tokenized = tokenize_corpus(train['cumm_text'])
        val_tokenized = tokenize_corpus(val['cumm_text'])

        MAX_DF = 0.8 #во скольких документах встречаеться слово
        MIN_COUNT = 5 # сколько раз слово встречаеться в тексте


        vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
        UNIQUE_WORDS_N = len(vocabulary)    
        VECTORIZATION_MODE = 'tfidf'
        train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

        train_vectors_gt100 = np.array([get_vectors_gt100(i) for i in train_tokenized])
        val_vectors_gt100 = np.array([get_vectors_gt100(i) for i in val_tokenized])

        model_t.most_similar(positive=['инвестор', 'рынок'], topn=1)

        y_train = train['pnl_sign']
        X_train = train_vectors_gt100

        y_val = val['pnl_sign']
        X_val = val_vectors_gt100

        model = LogisticRegression()
        model.fit(X_train, y_train)
        print(model.score(X_val, y_val))
        print('данные:', files[:q])
        print('всего наблюдений:', df_with_pnl.shape[0])

  0%|                                                    | 0/11 [00:00<?, ?it/s]
3572it [00:00, 44948.90it/s]
  9%|████                                        | 1/11 [00:01<00:18,  1.90s/it]

0.6190476190476191
данные: ['cbonds.csv']
всего наблюдений: 3618



3679it [00:00, 47984.32it/s]
 18%|████████                                    | 2/11 [00:03<00:17,  2.00s/it]

0.42857142857142855
данные: ['cbonds.csv', 'themovchans.csv']
всего наблюдений: 3726



3679it [00:00, 43039.80it/s]
 27%|████████████                                | 3/11 [00:06<00:16,  2.04s/it]

...

0.5238095238095238
данные: ['cbonds.csv', 'themovchans.csv', 'headlines_QUANTS.csv', 'War_Wealth_Wisdom.csv', 'mmi.csv', 'vts.csv', 'signal.csv', '.gitignore', 'zgmail.csv']
всего наблюдений: 20494



0it [00:00, ?it/s]
5187it [00:00, 51866.34it/s]
10641it [00:00, 53433.07it/s]
16056it [00:00, 53756.75it/s]
21791it [00:00, 53609.81it/s]
 91%|███████████████████████████████████████    | 10/11 [00:40<00:05,  5.52s/it]

0.5238095238095238
данные: ['cbonds.csv', 'themovchans.csv', 'headlines_QUANTS.csv', 'War_Wealth_Wisdom.csv', 'mmi.csv', 'vts.csv', 'signal.csv', '.gitignore', 'zgmail.csv', 'rshb_invest.csv']
всего наблюдений: 22043



0it [00:00, ?it/s]
5210it [00:00, 52090.49it/s]
10667it [00:00, 53547.07it/s]
16061it [00:00, 53724.57it/s]
21965it [00:00, 53616.15it/s]
100%|███████████████████████████████████████████| 11/11 [00:46<00:00,  4.23s/it]

0.47619047619047616
данные: ['cbonds.csv', 'themovchans.csv', 'headlines_QUANTS.csv', 'War_Wealth_Wisdom.csv', 'mmi.csv', 'vts.csv', 'signal.csv', '.gitignore', 'zgmail.csv', 'rshb_invest.csv', 'Alfa_Wealth.csv']
всего наблюдений: 22218
